In [ ]:
import numpy as np
import pandas as pd
from gensim.parsing.preprocessing import remove_stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import gensim
from gensim.models import Word2Vec
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding
from tensorflow.keras.models import Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data =pd.read_csv('/content/drive/MyDrive/fake news dataset/train.csv')

In [ ]:
#data =pd.read_csv('/content/drive/MyDrive/Newfolder/Code_And_Data/fake-news/train.csv')

In [ ]:
#data =pd.read_csv('/content/drive/MyDrive/ML Project - Fake news Detection /Code_And_Data/fake-news/train.csv')

In [ ]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
data.fillna('', inplace =True)

In [ ]:
X = data['author']+ ' '+ data['title']+ ' ' + data['text']
Y = data['label']

#lower case conversion


In [ ]:
X = X.str.lower()

#remove stopwords


In [ ]:
X= X.apply(lambda x: remove_stopwords(x))

#remove punctuation

In [ ]:

X = X.str.replace('[^\w\s]',' ').str.replace(r"http\S+|www\S+|https\S+",' ').str.replace(r'\@\w+|\#', ' ')
X = X.str.replace(r'\b\w\b', ' ').str.replace(r'\s+', ' ')

In [ ]:
X_split = [text.split() for text in X]

In [ ]:
myw2vmodel = Word2Vec(sentences=X_split,size=300,window=5,min_count=1,workers=10)

In [ ]:
weights = myw2vmodel.wv.syn0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


In [ ]:
weights.shape

(182038, 300)

In [ ]:
mytokenizer = Tokenizer()
mytokenizer.fit_on_texts(X_split)

In [ ]:
vocab_size = len(mytokenizer.word_index)

In [ ]:
vocab_size

182038

In [ ]:
#text to sequence
X_seq = mytokenizer.texts_to_sequences(X_split) 

In [ ]:
#padding text sequences
X_pad = pad_sequences(X_seq, maxlen=2000)

In [ ]:
X_pad.shape

(20800, 2000)

In [ ]:
weights = np.append(weights,[np.zeros(300)],axis=0, )

In [ ]:
weights.shape

(182040, 300)

In [ ]:

input = Input(shape=(X_pad.shape[1],))
layer = Embedding(weights.shape[0], weights.shape[1],weights=[weights], input_length=2000,trainable=False)(input)
layer= Conv1D(32, 3, activation='relu')(layer)
layer= MaxPooling1D(3)(layer)
layer= Conv1D(64, 3, activation='relu')(layer)
layer = MaxPooling1D(3)(layer)
layer = Conv1D(128, 3, activation='relu')(layer)
layer = GlobalMaxPooling1D()(layer6)
layer = Dense(1, activation='sigmoid')(layer)

model = Model(input, layer)


In [ ]:
from keras import backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy',f1_m,precision_m, recall_m]
)


print('Training model...')
r = model.fit(
  X_pad,
  Y,
  epochs=5,
  validation_split=0.3
)

Training model...
Epoch 1/5
455/455 [==============================] - 224s 489ms/step - loss: 0.2162 - accuracy: 0.9001 - f1_m: 0.8905 - precision_m: 0.8910 - recall_m: 0.9100 - val_loss: 0.1218 - val_accuracy: 0.9534 - val_f1_m: 0.9520 - val_precision_m: 0.9612 - val_recall_m: 0.9458
Epoch 2/5
455/455 [==============================] - 218s 479ms/step - loss: 0.0750 - accuracy: 0.9717 - f1_m: 0.9709 - precision_m: 0.9640 - recall_m: 0.9803 - val_loss: 0.0780 - val_accuracy: 0.9724 - val_f1_m: 0.9717 - val_precision_m: 0.9642 - val_recall_m: 0.9810
Epoch 3/5
455/455 [==============================] - 217s 477ms/step - loss: 0.0431 - accuracy: 0.9840 - f1_m: 0.9836 - precision_m: 0.9804 - recall_m: 0.9882 - val_loss: 0.0702 - val_accuracy: 0.9753 - val_f1_m: 0.9745 - val_precision_m: 0.9806 - val_recall_m: 0.9702
Epoch 4/5
455/455 [==============================] - 216s 476ms/step - loss: 0.0251 - accuracy: 0.9903 - f1_m: 0.9904 - precision_m: 0.9888 - recall_m: 0.9927 - val_loss: 0.06